In [1]:
from gensim import corpora, models
import pandas as pd
import numpy as np
import sys
from collections import Counter
sys.path.append("c:/users/david/desktop/sandbox/lsi-tagger")
from model.tag_extractor import TagExtractor
from model.text_cleaner import TextCleaner
import itertools

%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv('data/h&m_kaggle_products.csv')
df = df[~df.isnull().any(axis=1)]
print(df.shape)
df.head(2)

(105126, 25)


,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.


In [3]:
documents = df['detail_desc'].drop_duplicates().values

te = TagExtractor(word_count_min=1, word_length_min=2, num_lsi_topics=200)
te.fit(documents)

100%|█████████████████████████████████████████████████████████████████████████| 43404/43404 [00:04<00:00, 10618.35it/s]


In [4]:
input_tags, candidate_tags = te.extract_tags_and_rank(ranked_inputs=[
                             'Jersey top with narrow shoulder straps.', # Input
                             'Tights with built-in support to lift the bottom. Black in 30 denier and light amber in 15 denier.',
                             'Semi shiny tights that shape the tummy, thighs and calves while also encouraging blood circulation in the legs. Elasticated waist.',
                             'Jersey top with narrow shoulder straps.',
                             'uygnhjkl'
                          ], 
                          n_input_tags=10, n_candidate_tags=5)

input_tags, candidate_tags

([('narrow', 1), ('shoulder', 1), ('straps', 1), ('jersey', 1)],
 [{},
  {},
  {'narrow': 1.0831158879417275,
   'shoulder': 1.0500825729187626,
   'straps': 0.9355821979421521,
   'jersey': 0.5762472925840754},
  {}])

In [5]:
new_candidate_ranking = te.extract_tags_and_rank(ranked_inputs=candidate_tags, selected_tags=['jersey'])
new_candidate_ranking

[2, 0, 1, 3]

In [9]:
te.save()

In [10]:
te.__dict__.keys()

dict_keys(['word_count_min', 'word_length_min', 'num_lsi_topics', 'tc', 'problem_docs', 'doc2ind', 'dictionary', 'corpus', 'tfidf', 'corpus_tfidf', 'lsi_model', 'corpus_lsi', 'lsi_topic_matrix', 'tc_word_counts'])

In [14]:
te = TagExtractor()
te.load()

In [15]:
input_tags, candidate_tags = te.extract_tags_and_rank(ranked_inputs=[
                             'Jersey top with narrow shoulder straps.', # Input
                             'Tights with built-in support to lift the bottom. Black in 30 denier and light amber in 15 denier.',
                             'Semi shiny tights that shape the tummy, thighs and calves while also encouraging blood circulation in the legs. Elasticated waist.',
                             'Jersey top with narrow shoulder straps.',
                             'uygnhjkl'
                          ], 
                          n_input_tags=10, n_candidate_tags=5)

input_tags, candidate_tags

([('narrow', 1), ('shoulder', 1), ('straps', 1), ('jersey', 1)],
 [{},
  {},
  {'narrow': 1.0831158879417275,
   'shoulder': 1.0500825729187626,
   'straps': 0.9355821979421521,
   'jersey': 0.5762472925840754},
  {}])

In [16]:
new_candidate_ranking = te.extract_tags_and_rank(ranked_inputs=candidate_tags, selected_tags=['jersey'])
new_candidate_ranking

[2, 0, 1, 3]

In [26]:
import nltk
nltk.download('averaged_perceptron_tagger')

def pos_filter(ngram):
    if ('-pron-' in ngram) or ('t' in ngram):
        return False
    acceptable_types = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    second_type = ('NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if (tags[0][1] in acceptable_types) and (tags[1][1] in second_type):
        return True
    else:
        return False

def get_bigrams(tokens, bigram_freq_min=20):
    # Create bigrams
    bigram_finder = nltk.collocations.BigramCollocationFinder.from_words(tokens)

    # Filter bigrams by minimum frequency
    bigram_finder.apply_freq_filter(bigram_freq_min)

    # Calculate PMI
    bigrams_df = pd.DataFrame(list(bigram_finder.score_ngrams(nltk.collocations.BigramAssocMeasures().pmi)), 
                            columns=['bigram','PMI']).sort_values(by='PMI', ascending=False)
    bigrams_df = bigrams_df[bigrams_df['PMI'] > 1]

    # Filter bigrams based on POS
    bigrams_df = bigrams_df[bigrams_df['bigram'].apply(lambda x: pos_filter(x))]

    return bigrams_df

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\David\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [21]:
tc = TextCleaner()
cleaned_documents = tc.fit_transform(documents)

100%|██████████████████████████████████████████████████████████████████████████| 43404/43404 [00:04<00:00, 9742.18it/s]


In [34]:
bigrams_df = get_bigrams(list(itertools.chain(*cleaned_documents)), bigram_freq_min=200)
print(len(bigrams_df))
bigrams_df.head()

288


,bigram,PMI
1,"(room, movement)",11.471851
2,"(fuller, cleavage)",11.049920
3,"(broderie, anglaise)",10.963505
5,"(studio, collection)",10.835181
7,"(removable, inserts)",10.545574


In [36]:
bigrams_df.sample(10)

,bigram,PMI
239,"(soft, fine)",5.025012
422,"(waist, zip)",3.005565
247,"(cotton, twill)",4.951753
294,"(soft, organic)",4.472882
452,"(buttons, cuffs)",2.647960
428,"(stretch, weave)",2.986931
216,"(edge, hems)",5.319927
362,"(collar, zip)",3.710265
15,"(nursing, access)",10.294552
171,"(zip, chin)",5.984406


In [42]:
bigram_dict = bigrams_df['bigram'].head().apply(lambda x: {f'{x[0]},{x[1]}':f'{x[0]} {x[1]}'}).values
bigram_dict = {list(dict_.keys())[0]:list(dict_.values())[0] for dict_ in bigram_dict}
bigram_dict

{'room,movement': 'room movement',
 'fuller,cleavage': 'fuller cleavage',
 'broderie,anglaise': 'broderie anglaise',
 'studio,collection': 'studio collection',
 'removable,inserts': 'removable inserts'}

In [14]:
# TODO: Can give the user options of adjacent tags so that they can get new queries entirely?
# They can remove tags from the input document and add in other tags, like swapping out pants for jeans, etc.
# This results in a "new" input document

lsi_topic_matrix_T_normed = te.lsi_topic_matrix.T/np.linalg.norm(te.lsi_topic_matrix.T, axis=1)[:,None]

def lsi_corpus2vec(lsi_corpus_vec):
    vec = np.zeros(200)
    for ind, value in lsi_corpus_vec:
        vec[ind] = value
    return vec

tags = te.tc.transform([df['detail_desc'].drop_duplicates().values[777]])[0]
print(tags)
bow = te.dictionary.doc2bow(tags)
bow_vec = lsi_corpus2vec(te.lsi_model[te.tfidf[bow]])
bow_vec_normed = bow_vec/np.linalg.norm(bow_vec)
sims = bow_vec_normed.dot(lsi_topic_matrix_T_normed.T)
top_inds = np.argsort(-sims)
count = 0
for t in top_inds:
    tag = te.dictionary[t]
    if tag not in tags:
        print(tag)
        count += 1
    if count == 5:
        break

['single', 'breasted', 'jacket', 'stretch', 'weave', 'containing', 'wool', 'narrow', 'notch', 'lapels', 'decorative', 'buttonhole', 'fastening', 'buttons', 'chest', 'pocket', 'flap', 'pockets', 'inner', 'pockets', 'button', 'decorative', 'buttons', 'cuffs', 'single', 'vent', 'lined', 'skinny', 'fit', 'slightly', 'shorter', 'style', 'shapes', 'chest', 'tapers', 'sharply', 'waist', 'combined', 'narrow', 'shoulders', 'sleeves', 'creates', 'completely', 'fitted', 'silhouette']
narrower
tuxedo
special
linked
search
